**to do**
make pca on X values and color code with the model prediction see and compare with the color code being the truth value

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as metrics

import time

import os

import eli5

from sklearn import tree
import graphviz

from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots

import shap

import warnings

import scipy as sp

import lime
from lime import lime_tabular
import lightgbm as lgb
import seaborn as sns

import imblearn

In [4]:
os.listdir("../data/alge_data/")

['epoct_ezvir_05dec2018_dictionary.csv',
 'epoct_ezvir_05dec2018_labeled.csv',
 'epoct_ezvir_05dec2018_unlabeled - Edit.csv',
 'epoct_ezvir_05dec2018_unlabeled.csv',
 'epoct_ezvir_05dec2018_unlabeled_Jan.csv']

In [67]:
data = pd.read_csv("../data/alge_data/epoct_ezvir_05dec2018_unlabeled_Jan.csv")

In [38]:
data_labels = pd.read_csv("../data/alge_data/epoct_ezvir_05dec2018_dictionary.csv", encoding="unicode_escape")

In [96]:
X_names = ["dem_age_int",
           "dem_sex",  
            "signv_temp_d0",
            "symp_abdopain",
            "sign_restless_d0",
            "sign_danger",
            "symp_cough",
            "symp_feveronly",
            "symp_vomit",
            "signv_rr1_d0",
            "days_of_fever",           
]

y_names = ["lab_malaria_any_d0",]



In [97]:
X = data[X_names]
y = data[y_names]

X.fillna(X.median(), inplace=True)
y.fillna(y.median(), inplace=True)

In [98]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [99]:
model_forest = RandomForestClassifier(n_estimators=100, random_state=0).fit(train_X, np.ravel(train_y))

In [100]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [101]:
le = LabelEncoder()
mms = MinMaxScaler()

numerical=["dem_age_int","signv_temp_d0","signv_rr1_d0","days_of_fever"]
categorical=["dem_sex","symp_abdopain","sign_restless_d0","sign_danger","symp_cough","symp_feveronly","symp_vomit"]


data_numerical = pd.DataFrame(mms.fit_transform(X[numerical]), columns=numerical)
data_ml = X[categorical]
data_ml = data_ml.apply(le.fit_transform)



In [107]:
X_norm= pd.merge(data_ml, data_numerical, left_index=True, right_index=True)
train_X_norm, val_X_norm, train_y_norm, val_y_norm = train_test_split(X_norm, y, random_state=1)

model_forest_norm = RandomForestClassifier(n_estimators=100, random_state=0).fit(train_X_norm, np.ravel(train_y_norm))

In [110]:
from imblearn.over_sampling import RandomOverSampler

In [114]:
ros = RandomOverSampler(random_state=0, sampling_strategy=0.3)
X_ros, y_ros = ros.fit_sample(X_norm, y["lab_malaria_any_d0"])
train_X_ros, val_X_ros, train_y_ros, val_y_ros = train_test_split(X_ros, y_ros, random_state=1)


model_forest_norm_ros = RandomForestClassifier(n_estimators=100, random_state=0).fit(train_X_ros, np.ravel(train_y_ros))

In [116]:
validation_data = [val_X, val_X_norm, val_X_ros]
validation_label = [val_y, val_y_norm, val_y_ros]
models = [model_forest, model_forest_norm, model_forest_norm_ros]
names = ["Random forest: simple","Random forest:with normalized features","Random Forest with normalized and oversampled fatures"]

for idx, val in enumerate(models):
    model_preds = val.predict(validation_data[idx])
    accuracy = metrics.accuracy_score(model_preds,validation_label[idx])
    precision = metrics.precision_score(model_preds,validation_label[idx])
    recall = metrics.recall_score(model_preds,validation_label[idx])
    print("{} \naccuracy {} \nprecision {} \nrecall {}\n\n".format(names[idx],accuracy, precision, recall))

Random forest: simple 
accuracy 0.8959899749373433 
precision 0.10588235294117647 
recall 0.5625


Random forest:with normalized features 
accuracy 0.8909774436090225 
precision 0.09411764705882353 
recall 0.4444444444444444


Random Forest with normalized and oversampled fatures 
accuracy 0.9507494646680942 
precision 0.8724489795918368 
recall 0.890625


